In [1]:
from typing import Protocol, TypeVar, Union, Callable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv

import pathlib

In [2]:
# Simple derivative function
ArrayFunction = Callable[[np.ndarray], np.ndarray]

def derive(foo: ArrayFunction, X: np.ndarray, delta: float = 10e-4) -> np.ndarray:
    return (foo(X + delta) - foo(X - delta)) / (2 * delta)

def square(X: np.ndarray) -> np.ndarray:
    return np.square(X)

X = np.arange(12).reshape(3, 4)
print("Original Array: ", X, sep="\n")
print("Function Array: ", square(X), sep="\n")
print("Derived Array: ", derive(square, X), sep="\n")

Original Array: 
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
Function Array: 
[[  0   1   4   9]
 [ 16  25  36  49]
 [ 64  81 100 121]]
Derived Array: 
[[ 0.  2.  4.  6.]
 [ 8. 10. 12. 14.]
 [16. 18. 20. 22.]]


In [3]:
# Two Chained function derivative
from typing import List

ArrayFunction = Callable[[np.ndarray], np.ndarray]
Chain = List[ArrayFunction]

def derive(foo: ArrayFunction, X: np.ndarray, delta: float = 10e-4) -> np.ndarray:
    return (foo(X + delta) - foo(X - delta)) / (2 * delta)

def chain_derive_2(chain: Chain,
                   X: np.ndarray) -> np.ndarray:
    if len(chain) != 2:
        raise ValueError("Chain length should be two")
    
    f1 = chain[0]
    f2 = chain[1]

    f1x = f1(X)
    df1x_dx = derive(f1, X)
    df2x_dx = derive(f2, f1x)

    return df2x_dx * df1x_dx


def square(X: np.ndarray) -> np.ndarray:
    return np.square(X)

def cube(X: np.ndarray) -> np.ndarray:
    return X ** 3

chain = [square, cube]

X = np.arange(12).reshape(3, 4)
print("Original Array: ", X, sep="\n")
print("Derived Array: ", chain_derive_2(chain, X).astype(int), sep="\n")

Original Array: 
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
Derived Array: 
[[     0      6    192   1458]
 [  6144  18750  46656 100842]
 [196608 354294 600000 966306]]


In [4]:
# Chain derivation of three function
ArrayFunction = Callable[[np.ndarray], np.ndarray]
Chain = List[ArrayFunction]

def square(X: np.ndarray) -> np.ndarray:
    return np.square(X)

def cube(X: np.ndarray) -> np.ndarray:
    return X ** 3

def log(X: np.ndarray) -> np.ndarray:
    return np.log(X)

def derive(foo: ArrayFunction, X: np.ndarray, delta: float = 10e-4) -> np.ndarray:
    return (foo(X + delta) - foo(X - delta)) / (2 * delta)

def chain_derive_3(chain: Chain, X: np.ndarray) -> np.ndarray:
    if len(chain) != 3:
        raise ValueError("Length of chain is not equal to 3")
    
    f1 = chain[0]
    f2 = chain[1]
    f3 = chain[2]

    f1x = f1(X)
    f2f1x = f2(f1x)
    df1x_dx = derive(f1, X)
    df2f1_dx = derive(f2, f1x)
    df3x_dx = derive(f3, f2f1x)

    return df3x_dx * df2f1_dx * df1x_dx

chain = [square, cube, log]

X = np.arange(1, 13).reshape(3, 4)
print("Original Array: ", X, sep="\n")
print("Derived Array: ", chain_derive_3(chain, X), sep="\n")

Original Array: 
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
Derived Array: 
[[6.000004   3.00000006 2.00000001 1.5       ]
 [1.2        1.00000002 0.85714283 0.75000012]
 [0.66666688 0.60000005 0.54545442 0.4999992 ]]


In [5]:
# Derive functions with multiple parameters
ArrayFunction = Callable[[np.ndarray], np.ndarray]
Chain = List[ArrayFunction]

def square(X: np.ndarray) -> np.ndarray:
    return np.square(X)

def multiple_inputs_with_backward_derive(X: np.ndarray, 
                                         Y: np.ndarray, 
                                         sigma: ArrayFunction) -> np.ndarray:
    a = X + Y
    sigmaa_da = derive(sigma, a)
    daxy_dx = 1
    daxy_dy = 1
    
    return sigmaa_da * daxy_dx, sigmaa_da * daxy_dy

X = np.arange(1, 13).reshape(3, 4)
Y = np.arange(13, 25).reshape(3, 4)
print("Original Arrays: ", X, Y, sep="\n")
derived_arrays = multiple_inputs_with_backward_derive(X, Y, square)
print("Derived Array 1: ", derived_arrays[0], sep="\n")
print("Derived Array 2: ", derived_arrays[1], sep="\n")

Original Arrays: 
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
[[13 14 15 16]
 [17 18 19 20]
 [21 22 23 24]]
Derived Array 1: 
[[28. 32. 36. 40.]
 [44. 48. 52. 56.]
 [60. 64. 68. 72.]]
Derived Array 2: 
[[28. 32. 36. 40.]
 [44. 48. 52. 56.]
 [60. 64. 68. 72.]]


In [6]:
# Derivatives of vector function

ArrayFunction = Callable[[np.ndarray], np.ndarray]

def derive(foo: ArrayFunction, X: np.ndarray, delta: float = 10e-4) -> np.ndarray:
    return (foo(X + delta) - foo(X - delta)) / (2 * delta)

def vector_derive(X: np.ndarray, W: np.ndarray, sigma: ArrayFunction) -> np.ndarray:
    if X.shape[1] != W.shape[0]:
        raise ValueError("Shape mismatching for vector product")

    N = X @ W

    dN_dX = W.T; dN_dW = X.T

    dsigma_dN = derive(sigma, N)

    return dsigma_dN @ dN_dX, dN_dW @ dsigma_dN 

X = np.arange(1, 5).reshape(1, 4)
W = np.arange(6, 10).reshape(4, 1)
print("Original Arrays: ", X, W, sep="\n")
derived_arrays = vector_derive(X, W, square)
print("Derived Array 1: ", derived_arrays[0], sep="\n")
print("Derived Array 2: ", derived_arrays[1], sep="\n")

Original Arrays: 
[[1 2 3 4]]
[[6]
 [7]
 [8]
 [9]]
Derived Array 1: 
[[ 960.         1120.00000001 1280.00000001 1440.00000001]]
Derived Array 2: 
[[160.]
 [320.]
 [480.]
 [640.]]
